# ANÁLISIS DE UN DATASET REAL ANONIMIZADO

Utilizando los datos recogidos en el fichero de datos abiertos de PDI de la UAM (https://www.universidata.es/datasets/uam-personal-pdi) correspondiente al año 2019, se realiza, para cada campo del fichero exceptuando aquellos cuyo nombre empieza por "cod_" "lat_" y "lon_", lo siguiente:
* Inferir el valor más probable de cada campo para el caso del profesor Ortigosa
* Calcular el nivel de certeza (%) de que cada valor inferido corresponda realmente al profesor Ortigosa
* Una breve descripción del proceso de inferencia utilizado en cada caso


Para ello, se asegura que:
* El profesor Ortigosa estaba en 2019 en la UAM (=aparece en el fichero)
* El profesor Ortigosa era de género masculino en 2019
* El profesor Ortigosa pertenecía al Departamento de Ingeniería Informática en 2019

In [1]:
import pandas as pd
import numpy as np

### Preprocesamiento del dataset

In [2]:
data_path = 'uam-personal-pdi-2019-anonimizado.csv'
data = pd.read_csv(data_path)
remove = ['cod_', 'lat_','lon_']
index = [i for i in range(1, data.shape[1]) if data.columns[i][:4] not in remove]
data = data.iloc[:,index]

In [3]:
data.head()

,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,anio_nacimiento,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal
0,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,1963,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,1996.0,2003.0,7,4,2,NaN,N
1,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,1988,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,1,0,0,NaN,N
2,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,POZUELO DE ALARCÓN,Mujer,1991,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,0,0,0,NaN,N
3,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,ALCOBENDAS,Mujer,1972,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,6,3,0,NaN,N
4,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,MADRID,Mujer,1987,...,Departamento,"Departamento de Educación Física, Deporte y Mo...",Didáctica de la Expresión Corporal,NaN,NaN,0,0,0,NaN,N


## Información sobre el proceso de anonimización

La información sobre el proceso de anonimización se encuentra disponible en:
- https://dimetrical.atlassian.net/wiki/spaces/UNC/pages/515571713/Procesos+de+Anonimizaci+n#2.1.3-Ejemplo
- https://dimetrical.atlassian.net/wiki/spaces/UNC/pages/490799108/Dataset+Personal+Docente+e+Investigador+PDI

Dado que los campos pivote son 'des_unidad_responsable' y 'des_genero' y conocemos los valores de Ortigosa para ambos campos, esto es,
* Departamento de Ingeniería Informática
* Hombre

seleccionamos aquellas filas para las que se cumplan las anteriores restricciones sin tener pérdida de información.

In [4]:
data_ortigosa = data.loc[(data['des_unidad_responsable'] == 'Departamento de Ingeniería Informática') & (data['des_genero']=='Hombre')]
data_ortigosa.head()

,des_universidad,anio,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia,des_genero,anio_nacimiento,...,des_tipo_unidad_responsable,des_unidad_responsable,des_area_conocimiento,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios,num_tesis,ind_investigador_principal
149,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,COLMENAR VIEJO,Hombre,1972,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,6,3,0,1.0,N
150,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,"BOALO, EL",Hombre,1967,...,Departamento,Departamento de Ingeniería Informática,Ciencia de la Computación e Inteligencia Artif...,1995.0,2002.0,7,5,5,NaN,N
151,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,COLMENAR VIEJO,Hombre,1967,...,Departamento,Departamento de Ingeniería Informática,Ciencia de la Computación e Inteligencia Artif...,1982.0,1985.0,12,6,6,NaN,N
152,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,ALCOBENDAS,Hombre,1985,...,Departamento,Departamento de Ingeniería Informática,Ciencia de la Computación e Inteligencia Artif...,NaN,NaN,2,0,0,NaN,N
153,Universidad Autónoma de Madrid,2019,España,Europa,Europa meridional,Madrid,Madrid,LEGANÉS,Hombre,1965,...,Departamento,Departamento de Ingeniería Informática,Lenguajes y Sistemas Informáticos,NaN,NaN,8,4,2,NaN,N


A continuación, se inferirá el valor más probable de cada campo utilizando información del propio dataset y utilizando distintas fuentes externas. Dado que las permutaciones en el proceso de anonimización se mantienen para cada bloque, la confianza de inferir el valorm más probable dependerá del resto de campos que pertenezcan al mismo bloque.

### Bloque 1

En un principio conocemos esta información ya que hemos descargado el dataset que contiene los datos detallados sobre el personal Docente e Investigador de la Universidad Autónoma de Madrid en el año 2019. Aún así, haremos inferencia sobre el dataset para ver si coincide.

In [6]:
bloque_1 = data_ortigosa[['des_universidad','anio']]
bloque_1.head()

,des_universidad,anio
149,Universidad Autónoma de Madrid,2019
150,Universidad Autónoma de Madrid,2019
151,Universidad Autónoma de Madrid,2019
152,Universidad Autónoma de Madrid,2019
153,Universidad Autónoma de Madrid,2019


In [7]:
bloque_1_counts = bloque_1.value_counts(normalize=True, dropna=False)
bloque_1_counts

des_universidad                 anio
Universidad Autónoma de Madrid  2019    1.0
dtype: float64

En la celda anterior se calculan las probabilidades de obtener cada par de valores (probabilidad conjunta) de los campos que pertenecen al primer bloque. De esta manera, los campos más probables para este bloque serán lo que maximicen la probabilidad conjunta.

Como esperábamos, solo existe una posibilidad para la que se obtiene una probabilidad del 100%. Por tanto, inferimos con una confianza del 100% que el profesor Ortigosa era docente en la **Universidad Autónoma de Madrid** en el año **2019*.

### Bloque 2

In [8]:
bloque_2 = data_ortigosa[['des_pais_nacionalidad','des_continente_nacionalidad', 'des_agregacion_paises_nacionalidad']]
bloque_2.head()

,des_pais_nacionalidad,des_continente_nacionalidad,des_agregacion_paises_nacionalidad
149,España,Europa,Europa meridional
150,España,Europa,Europa meridional
151,España,Europa,Europa meridional
152,España,Europa,Europa meridional
153,España,Europa,Europa meridional


In [9]:
bloque_2_counts = bloque_2.value_counts(normalize=True, dropna=False)
bloque_2_counts

des_pais_nacionalidad  des_continente_nacionalidad  des_agregacion_paises_nacionalidad
España                 Europa                       Europa meridional                     0.982143
Italia                 Europa                       Europa meridional                     0.017857
dtype: float64

A la vista de los resultados obtenidos, inferimos con una confianza del 100% que Álvaro Ortigosa tiene una nacionalidad **Europea** de la **Europa meridional**. Respecto al país particular, las dos únicas posibilidades son que tenga nacionalidad Española o Italiana. Dado que lo más probable es que tenga nacionalidad española, concluimos con una confianza del 98.21% que tiene nacionalidad **española**. 

Además, en su perfil de Linkedin https://es.linkedin.com/in/alvaroortigosa no aparece ninguna relación con Italia.

### Bloque 3

In [10]:
bloque_3 =  data_ortigosa[['des_comunidad_residencia','des_provincia_residencia', 'des_municipio_residencia']]
bloque_3.head()

,des_comunidad_residencia,des_provincia_residencia,des_municipio_residencia
149,Madrid,Madrid,COLMENAR VIEJO
150,Madrid,Madrid,"BOALO, EL"
151,Madrid,Madrid,COLMENAR VIEJO
152,Madrid,Madrid,ALCOBENDAS
153,Madrid,Madrid,LEGANÉS


In [11]:
bloque_3_counts = bloque_3.value_counts(normalize=True, dropna=False)
bloque_3_counts

des_comunidad_residencia  des_provincia_residencia  des_municipio_residencia  
Madrid                    Madrid                    MADRID                        0.517857
                                                    COLMENAR VIEJO                0.107143
                                                    ALCOBENDAS                    0.089286
                                                    SAN SEBASTIÁN DE LOS REYES    0.089286
                                                    ALCALÁ DE HENARES             0.017857
                                                    ALCORCÓN                      0.017857
                                                    ALGETE                        0.017857
                                                    BOALO, EL                     0.017857
                                                    COLMENAREJO                   0.017857
                                                    GALAPAGAR                     0.017857
           

Inferimos con una probabilidad del 100% que Ortigosa reside en la **Comunidad de Madrid**. Respecto al municipio, con una confianza del 51.7% podríamos concluir que Ortigosa pertenece a Madrid o con una confianza del 48.3% que reside en uno de estos municipios (COLMENAR VIEJO, ALCOBENDAS, SAN SEBASTIÁN DE LOS REYES...).

### Bloque 4

Leyendo la bibliografía de Álvaro en https://ieeexplore.ieee.org/author/37668186700, encontramos que nació en 1968.

In [12]:
bloque_4 = data_ortigosa[['anio_nacimiento']].loc[data_ortigosa.anio_nacimiento == 1968]
bloque_4.head()

,anio_nacimiento
183,1968
184,1968
193,1968


Existen 3 profesores que nacieron en 1968. Como este bloque sólo contiene un campo, no podemos inferir ninguna conclusión más.

### Bloque 5

In [13]:
bloque_5 = data_ortigosa[['des_tipo_personal', 'des_categoria_cuerpo_escala', 'des_tipo_contrato', 'des_dedicacion', 'num_horas_semanales_tiempo_parcial', 'des_situacion_administrativa']]
bloque_5 = bloque_5.loc[bloque_5.des_categoria_cuerpo_escala == 'Profesor Asociado']
bloque_5.head()

,des_tipo_personal,des_categoria_cuerpo_escala,des_tipo_contrato,des_dedicacion,num_horas_semanales_tiempo_parcial,des_situacion_administrativa
155,Personal laboral,Profesor Asociado,Contrato de Duración Determinada,Dedicación a Tiempo Parcial,8.0,Servicio Activo
158,Personal laboral,Profesor Asociado,Contrato de Duración Determinada,Dedicación a Tiempo Parcial,8.0,Servicio Activo
159,Personal laboral,Profesor Asociado,Contrato de Duración Determinada,Dedicación a Tiempo Parcial,12.0,Servicio Activo
163,Personal laboral,Profesor Asociado,Contrato de Duración Determinada,Dedicación a Tiempo Parcial,8.0,Servicio Activo
164,Personal laboral,Profesor Asociado,Contrato de Duración Determinada,Dedicación a Tiempo Parcial,8.0,Servicio Activo


En Linkedin (https://es.linkedin.com/in/alvaroortigosa), Ortigosa figuraba como profesor asociado en 2019.

In [14]:
bloque_5_counts = bloque_5.value_counts(normalize=True, dropna=False)
bloque_5_counts

des_tipo_personal  des_categoria_cuerpo_escala  des_tipo_contrato                 des_dedicacion               num_horas_semanales_tiempo_parcial  des_situacion_administrativa
Personal laboral   Profesor Asociado            Contrato de Duración Determinada  Dedicación a Tiempo Parcial  8.0                                 Servicio Activo                 0.909091
                                                                                                               12.0                                Servicio Activo                 0.090909
dtype: float64

Con una probabilidad del 90.9% inferimos de los datos que Ortigosa en 2019 era Personal laboral, profesor asociado, con un contrato de Duración Determinada y una dedicación a Tiempo Parcial de 12.0 horas semanales y un servicio activo.

### Bloque 6

In [15]:
bloque_6 = data_ortigosa[['ind_cargo_remunerado']]
bloque_6.head()

,ind_cargo_remunerado
149,N
150,N
151,N
152,N
153,N


In [16]:
bloque_6_counts = bloque_6.value_counts(normalize=True, dropna=False)
bloque_6_counts

ind_cargo_remunerado
N                       0.821429
S                       0.178571
dtype: float64

De los resultados, podríamos inferir que Ortigosa que no ocupaba en 2019 ningún cargo unipersonal remunerado económicamente en la Universidad con una confianza del 82.14%. Sin embargo, en el portal científico de la Autónoma https://portalcientifico.uam.es/ipublic/agent-personal/profile/iMarinaID/04-261195 figura como Director del INST. CIENCIAS FORENSES Y DE LA SEGURIDAD. Por tanto, concluimos que si ocupa un cargo unipersonal con una confianza del 100%.

### Bloque 7

De la fuente https://portalcientifico.uam.es/ipublic/agent-personal/profile/iMarinaID/04-261195 sabemos que realizó el Doctorado en España en la Universidad Autónoma de Madrid en Enero de 1997 y que lo terminó en Enero del 2000. Además, en la misma fuente indica que lo realizó en Ingeniería Informática y su tesis doctoral se titula: Método para la aplicación de documentación inteligente de frameworks orientados a objetos.

In [17]:
bloque_7 = data_ortigosa[['des_titulo_doctorado', 'des_pais_doctorado', 'des_continente_doctorado', 'des_agregacion_paises_doctorado', 'des_universidad_doctorado', 'anio_lectura_tesis', 'anio_expedicion_titulo_doctor', 'des_mencion_europea']]
bloque_7 = bloque_7.loc[(bloque_7.des_titulo_doctorado == "Uno") &
            (bloque_7.anio_lectura_tesis == 2000) &
            (bloque_7.des_universidad_doctorado == "Universidad Autónoma de Madrid")]
bloque_7.head()

,des_titulo_doctorado,des_pais_doctorado,des_continente_doctorado,des_agregacion_paises_doctorado,des_universidad_doctorado,anio_lectura_tesis,anio_expedicion_titulo_doctor,des_mencion_europea
171,Uno,España,Europa,Europa meridional,Universidad Autónoma de Madrid,2000.0,2000.0,No
185,Uno,España,Europa,Europa meridional,Universidad Autónoma de Madrid,2000.0,2000.0,No


In [18]:
bloque_7_counts = bloque_7.value_counts(normalize=True, dropna=False)
bloque_7_counts

des_titulo_doctorado  des_pais_doctorado  des_continente_doctorado  des_agregacion_paises_doctorado  des_universidad_doctorado       anio_lectura_tesis  anio_expedicion_titulo_doctor  des_mencion_europea
Uno                   España              Europa                    Europa meridional                Universidad Autónoma de Madrid  2000.0              2000.0                         No                     1.0
dtype: float64

El resto de campos podemos inferirmos de los datos: Ortigosa expidió en título en el 2000 y no consiguió mención europea con una confianza del 100%.

### Grupo 8

In [19]:
bloque_8 = data_ortigosa[['des_tipo_unidad_responsable', 'des_area_conocimiento']]
bloque_8.head()

,des_tipo_unidad_responsable,des_area_conocimiento
149,Departamento,Lenguajes y Sistemas Informáticos
150,Departamento,Ciencia de la Computación e Inteligencia Artif...
151,Departamento,Ciencia de la Computación e Inteligencia Artif...
152,Departamento,Ciencia de la Computación e Inteligencia Artif...
153,Departamento,Lenguajes y Sistemas Informáticos


In [20]:
bloque_8_counts = bloque_8.value_counts(normalize=True, dropna=False)
bloque_8_counts

des_tipo_unidad_responsable  des_area_conocimiento                              
Departamento                 Ciencia de la Computación e Inteligencia Artificial    0.535714
                             Lenguajes y Sistemas Informáticos                      0.464286
dtype: float64

En https://portalcientifico.uam.es/ipublic/agent-personal/profile/iMarinaID/04-261195, figura que su área de conocimiento es Lenguajes y Sistemas Informáticos. Por tanto concluimos que en 2019 trabajaba en el Departamento de Lenguajes y Sistemas Informáticos con una confianza del 100%.

### Bloque 9:

En https://portalcientifico.uam.es/ipublic/agent-personal/profile/iMarinaID/04-261195  figura que Ortigosa tenía en 2016 un total de 3 quinquenios y que en 2017 tenía 4 sexenios.

In [21]:
bloque_9 = data_ortigosa[['anio_incorporacion_ap', 'anio_incorpora_cuerpo_docente', 'num_trienios', 'num_quinquenios', 'num_sexenios']]
bloque_9 = bloque_9.loc[(bloque_9.num_sexenios == 4) &
            (bloque_9.num_quinquenios == 3)]
bloque_9.head()

,anio_incorporacion_ap,anio_incorpora_cuerpo_docente,num_trienios,num_quinquenios,num_sexenios
169,NaN,NaN,5,3,4
187,NaN,NaN,7,3,4
204,NaN,NaN,6,3,4


In [22]:
bloque_9_counts = bloque_9.value_counts(normalize=True, dropna=False)
bloque_9_counts

anio_incorporacion_ap  anio_incorpora_cuerpo_docente  num_trienios  num_quinquenios  num_sexenios
NaN                    NaN                            5             3                4               0.333333
                                                      6             3                4               0.333333
                                                      7             3                4               0.333333
dtype: float64

No tenemos evidencia estadística del número de trienios ni del año de incorporación al cuerpo docente y a la administración pública.

### Bloque 10

In [23]:
bloque_10 = data_ortigosa[['num_tesis']]
bloque_10.head()

,num_tesis
149,1.0
150,NaN
151,NaN
152,NaN
153,NaN


In [24]:
bloque_10_counts = bloque_10.value_counts(normalize=True, dropna=False)
bloque_10_counts

num_tesis
NaN          0.875000
1.0          0.089286
2.0          0.035714
dtype: float64

En https://portalcientifico.uam.es/ipublic/agent-personal/profile/iMarinaID/04-261195, aparece que Ortigosa ha dirigido 3 tesis pero no aparece ningua en 2019. Por tanto, suponiendo que Nan significa que no ha dirigido ninguna tesis, inferimos con una confianza del 87.5% que Álvaro no dirigió ninguna tesis en 2019.

### Bloque 11:

In [25]:
bloque_11 = data_ortigosa[['ind_investigador_principal']]
bloque_11.head()

,ind_investigador_principal
149,N
150,N
151,N
152,N
153,N


In [26]:
bloque_11_counts = bloque_11.value_counts(normalize=True, dropna=False)
bloque_11_counts

ind_investigador_principal
N                             0.785714
S                             0.214286
dtype: float64

En https://www.altmetric.com/details/59878968 se muestra que Ortigosa desarrolló este proyecto como investigador principal en 2019.